In [1]:
# LSTM for international airline passengers problem with regression framing
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [4]:
# fix random seed for reproducibility
tf.random.set_seed(7)

# load the dataset
dataframe = pd.read_csv('../data/HistoryData.csv', engine='python') #df["Close"]
dataset = dataframe[['Close']].values
dataset = dataset.astype('float32')
dataset[:5]

array([[20050.64],
       [20056.95],
       [20019.31],
       [19920.05],
       [20033.94]], dtype=float32)

In [5]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
dataset[:5]

array([[0.3956151 ],
       [0.39715004],
       [0.3879919 ],
       [0.36384058],
       [0.3915515 ]], dtype=float32)

In [6]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [7]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

2022-10-15 16:17:18.504410: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-10-15 16:17:18.507627: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-15 16:17:18.715851: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/100
119/119 - 1s - loss: 0.1706
Epoch 2/100
119/119 - 0s - loss: 0.0818
Epoch 3/100
119/119 - 0s - loss: 0.0396
Epoch 4/100
119/119 - 0s - loss: 0.0296
Epoch 5/100
119/119 - 0s - loss: 0.0272
Epoch 6/100
119/119 - 0s - loss: 0.0253
Epoch 7/100
119/119 - 0s - loss: 0.0232
Epoch 8/100
119/119 - 0s - loss: 0.0211
Epoch 9/100
119/119 - 0s - loss: 0.0189
Epoch 10/100
119/119 - 0s - loss: 0.0169
Epoch 11/100
119/119 - 0s - loss: 0.0150
Epoch 12/100
119/119 - 0s - loss: 0.0133
Epoch 13/100
119/119 - 0s - loss: 0.0116
Epoch 14/100
119/119 - 0s - loss: 0.0102
Epoch 15/100
119/119 - 0s - loss: 0.0087
Epoch 16/100
119/119 - 0s - loss: 0.0076
Epoch 17/100
119/119 - 0s - loss: 0.0066
Epoch 18/100
119/119 - 0s - loss: 0.0059
Epoch 19/100
119/119 - 0s - loss: 0.0054
Epoch 20/100
119/119 - 0s - loss: 0.0050
Epoch 21/100
119/119 - 0s - loss: 0.0047
Epoch 22/100
119/119 - 0s - loss: 0.0045
Epoch 23/100
119/119 - 0s - loss: 0.0044
Epoch 24/100
119/119 - 0s - loss: 0.0043
Epoch 25/100
119/119 - 0s

In [8]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [27]:
ms_in_hour = 3.6e6
last_date = dataframe.iloc[-1,0]

X = testX[-1].reshape(1,1,1)

y = np.array([])
dates = np.array([])
predictions = 6
n = 0

while(n<predictions):    
    y_pred = model.predict(X)
    y = np.append(y, y_pred)

    next_date = (4 * ms_in_hour) + last_date
    dates = np.append(dates, next_date)    

    X=y_pred.reshape(1,1,1)
    last_date = next_date
    n+=1

In [30]:
dataframe.iloc[0,0]

1661659200000

In [28]:
dates

array([1.6642656e+12, 1.6642800e+12, 1.6642944e+12, 1.6643088e+12,
       1.6643232e+12, 1.6643376e+12])

In [25]:
dates

array([1.6642656e+12, 1.6642800e+12, 1.6642944e+12, 1.6643088e+12,
       1.6643232e+12, 1.6643376e+12])

In [10]:
y = np.expand_dims(y, axis=0)
f = scaler.inverse_transform(y)
f = np.reshape(f,-1)

In [11]:
pred = pd.DataFrame(np.vstack((dates, f))).T
pred.columns = ["Time", "Forecasted Price"]
#pred["Time"] = pd.to_datetime(pred.Time, unit='ms')
pred

,Time,Forecasted Price
0,1.664266e+12,19246.234317
1,1.664280e+12,19287.364367
2,1.664294e+12,19324.063283
3,1.664309e+12,19357.020653
4,1.664323e+12,19386.784353
5,1.664338e+12,19413.795512


In [15]:
from pushbullet import PushBullet
import time
import os

In [16]:
# Get the access token from Pushbullet.com
access_token = ""
  
# Get the instance using access token
pb = PushBullet(access_token)

In [20]:
forecast_channel = pb.channels[0]
forecast_channel.push_note("123","body")

{}

In [31]:
pred.to_csv("../data/ForecastedPrices.csv")

In [24]:
pd.to_datetime(pred.Time, unit="ms") 

0   2022-09-27 08:00:00
1   2022-09-27 12:00:00
2   2022-09-27 16:00:00
3   2022-09-27 20:00:00
4   2022-09-28 00:00:00
5   2022-09-28 04:00:00
Name: Time, dtype: datetime64[ns]

In [40]:
body = pred.to_csv(header=False, index=False)

In [42]:
forecast_channel.push_note("123",body)

{}

In [45]:
p = [pred, "coin", pred]
for i in p:
    print(i)

           Time  Forecasted Price
0  1.664266e+12      19246.234317
1  1.664280e+12      19287.364367
2  1.664294e+12      19324.063283
3  1.664309e+12      19357.020653
4  1.664323e+12      19386.784353
5  1.664338e+12      19413.795512
coin
           Time  Forecasted Price
0  1.664266e+12      19246.234317
1  1.664280e+12      19287.364367
2  1.664294e+12      19324.063283
3  1.664309e+12      19357.020653
4  1.664323e+12      19386.784353
5  1.664338e+12      19413.795512


In [46]:
d = {
    "bitcoin": pred,
    "sol" : pred
}

for i in d.keys():
    print(type(d[i]))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [49]:
round(pred["Forecasted Price"],2)

0    19246.23
1    19287.36
2    19324.06
3    19357.02
4    19386.78
5    19413.80
Name: Forecasted Price, dtype: float64